In [1]:
%pwd

'c:\\Users\\user\\Desktop\\History-and-philosophy-chatbot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\user\\Desktop\\History-and-philosophy-chatbot'

In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter #for chunking

In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [6]:
# Extraction of Document
def load_pdf_file(data):
    loader = DirectoryLoader(
        data, glob="*.pdf", loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data="Data/")

In [13]:
print(f"Loaded {len(extracted_data)} documents.")

Loaded 1210 documents.


In [8]:
extracted_data[0]

Document(metadata={'source': 'Data\\Ayomide_Popoola_CV_AI_ML_Engineer.pdf', 'page': 0, 'page_label': '1'}, page_content='POPOOLA, AYOMIDE PETER, MBCS \nTel: +44 7586 367163 Email Address: popoolaayomide37@gmail.com  \nLinkedIn profile: https://www.linkedin.com/in/ayomide-popoola-60092a12a/  \nPortfolio: https://peteray-dev.github.io/Portfolio_DS/   GitHub profile: https://github.com/peteray-dev/ \n \nPROFESSIONAL SUMMARY software engineer compliance ml  \nInnovative AI & Machine Learning Engineer with expertise in developing, deploying, and optimising machine \nlearning models to drive measurable business impact. Proficient in generative AI, deep learning, natural language \nprocessing (NLP), large language models (LLMs), and high -performance computing (HPC). Adept at \nimplementing MLOps best practices, CI/CD pipelines, and cloud -based AI solutions to enhance operational \nefficiency. Passionate about collaborating with stakeholders to deliver innovative, data-driven products that e

In [9]:
# ~Chunking
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [10]:
text_chunks = text_split(extracted_data=extracted_data)

In [11]:
# text_chunks

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings



In [14]:
embeddings = download_hugging_face_embedding()

C:\Users\user\AppData\Local\Temp\ipykernel_30584\786653829.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


In [15]:
query_result = embeddings.embed_query("hello world")

len(query_result)

384

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [18]:
# print(OPENAI_API_KEY)

In [19]:
# from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
# from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "histphilbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [20]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name= index_name,
    embedding= embeddings
)

In [22]:
# load existing index

from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
retriever = doc_search.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_doc = retriever.invoke("who is popoola?")

In [25]:
retrieved_doc

[Document(id='2112917b-c03b-4570-a130-7418733d0c5e', metadata={'page': 363.0, 'page_label': '364', 'source': 'Data\\World_History-book.pdf'}, page_content='When I was at Marcavillca, in the province of Xauxa, I asked the lord Guacarapora to explain it in such a\nway as that my mind might be satisfied, and that I might be assured that it was true and accurate. He\nordered his servants to bring the quipus, and as this lord was a native, and a man of good\nunderstanding, he proceeded to make the thing clear to me. He told me to observe that all that he, for'),
 Document(id='a53f3ec5-67ac-4843-9dae-12b5fbd504ea', metadata={'page': 43.0, 'page_label': '44', 'source': 'Data\\phylosophy-book.pdf'}, page_content='podcast “Philosophy Bites” that is one of the most downloaded podcasts on academic topics. He also is an\neditor-in-chief of the online magazine Aeon. David Barnett, a former philosophy professor, founded the\ncompany PopSockets in 2012 after leaving academia. That company now employs

In [46]:
# to have a complete reply, i will integrate such that the query get the similarities
# fromm the knowledge based and then hits the llm (openai) along side the query itself
# the llm then process the 2

In [34]:
# print(OPENAI_API_KEY )

from langchain_openai import OpenAI
llm = OpenAI(temperature=0.3, max_tokens=500)

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know, and it is out of context. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [39]:
response = rag_chain.invoke({"input": "who is popoola?"})
print(response["answer"])



Popoola is a software engineer with expertise in developing, deploying, and optimizing machine learning models. He has a background in philosophy and has worked in academia before founding a successful company in 2012. He is also an editor-in-chief of an online magazine and has a popular podcast on academic topics. 
